In [ ]:
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 7.8 MB/s eta 0:00:00


In [ ]:
from qdrant_client import QdrantClient, models

In [ ]:
import os
from google.colab import userdata

client = QdrantClient("https://b9d13b33-842f-4f65-8d2b-de07f146ebea.europe-west3-0.gcp.cloud.qdrant.io", api_key = userdata.get('api_key'))

## Create a Collection

In [ ]:
collection_name = "my_first_collection"

client.create_collection(
    collection_name = collection_name,
    vectors_config = models.VectorParams(size = 4, distance = models.Distance.COSINE)
)

True

In [ ]:
# Retrieve and display the list of collections
collection_names = client.get_collections()
print("Existing collections: ", collection_names)

Existing collections:  collections=[CollectionDescription(name='my_first_collection')]


In [ ]:
# Define the vectors to be inserted
points = [
    models.PointStruct(
        id = 1,
        vector = [0.1, 0.2, 0.3, 0.4],
        payload = {"category": "example"}
    ),
    models.PointStruct(
        id = 2,
        vector = [0.2, 0.3, 0.4, 0.5],
        payload = {"category": "demo"}
    )
]

# Insert vectors into collection
client.upsert(
    collection_name = collection_name,
    points = points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

### How Similarity Search Works

In [ ]:
query_vector = [0.08, 0.14, 0.33, 0.28]

search_results = client.query_points(
    collection_name = collection_name,
    query = query_vector,
    limit = 1  # return the top 1 most similar vector
)

print("Search Results: ", search_results.points)

Search Results:  [ScoredPoint(id=1, version=1, score=0.97642946, payload={'category': 'example'}, vector=None, shard_key=None, order_value=None)]


In [ ]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="category",
    field_schema=models.PayloadSchemaType.KEYWORD, # "keyword" for exact matches
)

search_filter = models.Filter(
        must = [
            models.FieldCondition(
                key = "category",
                match = models.MatchValue(value = "example")
            )
        ]
    )

filtered_results = client.query_points(
    collection_name = collection_name,
    query = query_vector,
    query_filter = search_filter,
    limit = 1
)

print("Filtered search results: ", filtered_results)

Filtered search results:  points=[ScoredPoint(id=1, version=1, score=0.97642946, payload={'category': 'example'}, vector=None, shard_key=None, order_value=None)]
